In [ ]:
import streamlit as st
import requests
import networkx as nx
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

from urllib.parse import urljoin
from PIL import Image

# OpenAI API Key
import openai

api_key = "Api_KEY"
openai.api_key = api_key

from urllib.parse import urljoin, urlparse

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

def extract_links_from_page(url, page_content):
    links = []
    soup = BeautifulSoup(page_content, "html.parser")
    for link in soup.find_all("a"):
        href = link.get("href")
        if href:
            absolute_url = urljoin(url, href)
            links.append(absolute_url)
    return links


def create_graph(url, page_content, max_links=100):
    graph = nx.Graph()
    graph.add_node(url)

    links = extract_links_from_page(url, page_content)
    for link in links[:max_links]:
        graph.add_node(link)
        graph.add_edge(url, link)

    return graph


def plot_graph(graph):
    pos = nx.spring_layout(graph)
    nx.draw(graph, pos, with_labels=True, node_size=1000, font_size=10)
    plt.title("Web Sayfası Grafiği")
    st.pyplot(plt)


def main():
    st.title("Web Sayfası İndirme ve Graf Oluşturma")

    url = st.text_input("Lütfen web adresini girin", "")

    if st.button("Graf Oluştur"):
        if url:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    page_content = response.text

                    st.text("Graf Oluşturuluyor...")
                    graph = create_graph(url, page_content)
                    plot_graph(graph)
                    
                else:
                    st.error("Sayfa indirilemedi. Lütfen geçerli bir URL girin.")
            except Exception as e:
                st.error("Bir hata oluştu: {}".format(str(e)))
        else:
            st.warning("Lütfen bir URL girin.")


if __name__ == "__main__":
    main()
